In [1]:
# import functions
# import predictions
# from predictions import Prediction
# from features import Features
import mlp
from mlp import Classifier
from data import dataset as data
# import attacks
import torch
import pandas as pd
import numpy as np
import torch.nn as nn 
import matplotlib.pyplot as plt
import torch.nn.functional as F
import seaborn as sns 
import random
import pickle
import torch 
from sklearn.metrics import auc
from autoattack import AutoAttack
from torchvision.utils import save_image
import math
from matplotlib.backends.backend_pdf import PdfPages
# import train_model
import os
# from ensemble_whitebox import New
import timm 
import torchsummary
import sys
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from torchsummary import summary
import pickle

def countParams(model):
    total_params = sum(param.numel() for param in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Total Params: ", total_params)
    print("Trainable Params: ", trainable_params)

# 1. Training mlp modules

In [2]:
from mlp import Classifier

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = Classifier(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs): 
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/mlps/mlp_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 189/189 [00:27<00:00,  6.85it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.74
train_loss = 7.84
Epoch 2/20


100%|██████████| 189/189 [00:27<00:00,  6.91it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.84
train_loss = 0.37
Epoch 3/20


...

Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.12
Epoch 20/20


100%|██████████| 189/189 [00:46<00:00,  4.09it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.12
Testing


100%|██████████| 23/23 [00:04<00:00,  5.52it/s]


test_acc = 0.96
test_loss = 0.12
================= Block 11 Finished ==============


# 1.1 Training r_mlp modules

In [ ]:
from ReVIT.random_mlp import Classifier

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = Classifier(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs): 
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/r_mlps/r_mlp_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

# 1.2 Training Adversarial_mlp modules

In [3]:
from ReVIT.random_mlp import AdversarialClassifier as Classifier

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 10
lr = 1e-3
for index in range(12): 
    classifier = Classifier(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs): 
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            print(x.shape)
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/r_mlps/r_mlp_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/10


  0%|          | 0/189 [00:00<?, ?it/s]

torch.Size([30, 197, 768])


  0%|          | 0/189 [00:02<?, ?it/s]


RecursionError: maximum recursion depth exceeded while calling a Python object

# 2. Training CNN modules

In [2]:
from ReVIT.CNN import CNN

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = CNN(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/cnns/cnn_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 189/189 [00:11<00:00, 16.21it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.72
train_loss = 0.56
Epoch 2/20


100%|██████████| 189/189 [00:11<00:00, 16.03it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.80
train_loss = 0.42
Epoch 3/20


...

Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.11
Epoch 20/20


100%|██████████| 189/189 [00:25<00:00,  7.56it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.11
Testing


100%|██████████| 23/23 [00:03<00:00,  5.80it/s]

test_acc = 0.97
test_loss = 0.11
================= Block 11 Finished ==============


# 3. Training ResNet_FT

In [3]:
from ReVIT.ResnetFT import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/resnets_ft/resnet_ft_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 189/189 [00:11<00:00, 16.08it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.74
train_loss = 0.54
Epoch 2/20


100%|██████████| 189/189 [00:11<00:00, 16.04it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.84
train_loss = 0.37
Epoch 3/20


...

Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.12
Epoch 20/20


100%|██████████| 189/189 [00:27<00:00,  6.96it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.11
Testing


100%|██████████| 23/23 [00:04<00:00,  5.69it/s]


test_acc = 0.97
test_loss = 0.10
================= Block 11 Finished ==============


# 4. Training ResNet_TL

In [4]:
from ReVIT.ResnetTL import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/resnets_tl/resnet_tl_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 189/189 [00:11<00:00, 15.96it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.69
train_loss = 0.57
Epoch 2/20


100%|██████████| 189/189 [00:11<00:00, 16.26it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.76
train_loss = 0.50
Epoch 3/20


...

Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.94
train_loss = 0.17
Epoch 20/20


100%|██████████| 189/189 [00:26<00:00,  7.16it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.93
train_loss = 0.18
Testing


100%|██████████| 23/23 [00:03<00:00,  5.82it/s]


test_acc = 0.94
test_loss = 0.18
================= Block 11 Finished ==============


# 5. Training ResNet_FT_CNN

In [6]:
from ReVIT.ResnetFT_CNN import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/resnets_ft_cnn/resnet_ft_cnn_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 189/189 [00:12<00:00, 15.75it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.76
train_loss = 0.49
Epoch 2/20


100%|██████████| 189/189 [00:11<00:00, 16.01it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.83
train_loss = 0.38
Epoch 3/20


...

100%|██████████| 189/189 [00:27<00:00,  6.96it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.96
train_loss = 0.11
Testing


100%|██████████| 23/23 [00:04<00:00,  5.58it/s]


test_acc = 0.97
test_loss = 0.15
================= Block 11 Finished ==============


: 

# 6. Training ResNet_TL_CNN

In [5]:
from ReVIT.ResnetTL_CNN import ResNet18

# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']= '4'

epochs = 100
lr = 1e-3

vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
image_size = (224,224)
batch_size = 30

model = torch.load(vit).cuda()

for w in model.parameters(): 
    w.requires_grad = False

root_dir = "data/TB_data/"
data_loader, image_dataset = data.data_loader(root_dir=root_dir, batch_size= batch_size, image_size=image_size)


test_list = {f'block_{i}': [] for i in range(12)}
epochs = 20
lr = 1e-3
for index in range(12): 
    classifier = ResNet18(num_classes=2).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(),lr = lr)
    scheduler = StepLR(optimizer=optimizer, step_size=15, gamma=0.1, verbose=True)

    classifier.train()
    for epoch in range(epochs):
        train_acc = 0.0
        train_loss = 0.0 
        print(f'Epoch {epoch+1}/{epochs}')
        for image, label in tqdm(data_loader['train']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            # output = classifier(x[:,0,:])
            output = classifier(x[:,1:,:])
            prediction = torch.argmax(output , dim= -1)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = sum(prediction == label).float().item()/len(label)
            train_acc += acc 
            
        scheduler.step() 
        train_acc = train_acc/len(data_loader['train'])   
        train_loss = train_loss/len(data_loader['train'])
        print(f'train_acc= {train_acc:.2f}')
        print (f'train_loss = {train_loss:.2f}')    
    
    #Test loop 
    print('Testing')
    classifier.eval()
    test_acc = 0.0
    test_loss = 0.0

    with torch.no_grad(): 
        for image, label in tqdm(data_loader['test']):
            image = image.cuda()
            label = label.cuda()
            x = model.patch_embed(image)
            x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
            x = model.pos_drop(x + model.pos_embed)
            for i in range(index+1):
                x = model.blocks[i](x)
            output = classifier(x[:,1:,:])
            loss = criterion(output, label)
            acc = sum(torch.argmax(output, dim=-1) == label).float().item()/len(label)
            test_acc += acc
            test_loss += loss.item()
    test_acc = test_acc/len(data_loader['test'])
    test_loss = test_loss/len(data_loader['test'])
    print(f'test_acc = {test_acc:.2f}')
    print (f'test_loss = {test_loss:.2f}')
    test_list[f'block_{index}'].append(test_acc)
    print(f'================= Block {index} Finished ==============')
    
    save_pth = f'ReVIT/models/resnets_tl_cnn/resnet_tl_cnn_block_{index}_train_acc{train_acc:.2f}_test_acc{test_acc:.2f}.pth'
    torch.save(classifier, save_pth)

with open("accuracy_list_chest_mlp_cls_latest", "wb") as fp:   #Pickling
     pickle.dump(test_list, fp)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/20


100%|██████████| 189/189 [00:11<00:00, 16.38it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.70
train_loss = 0.57
Epoch 2/20


100%|██████████| 189/189 [00:11<00:00, 16.28it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
train_acc= 0.78
train_loss = 0.48
Epoch 3/20


...

Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.94
train_loss = 0.15
Epoch 20/20


100%|██████████| 189/189 [00:26<00:00,  7.12it/s]


Adjusting learning rate of group 0 to 1.0000e-04.
train_acc= 0.95
train_loss = 0.15
Testing


100%|██████████| 23/23 [00:04<00:00,  5.72it/s]


test_acc = 0.96
test_loss = 0.15
================= Block 11 Finished ==============


In [16]:
x = torch.randn(30, 196, 768)
# print(x.shape)

import torch.nn as nn
import torch

# This is what Faris told
class CNN(nn.Module):
    """
    CNN classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=128, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(in_features=3*3*64, out_features=128)
        self.relu3 = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, x):
        print(x.shape)
        # x = x.view(-1, 768, 14, 14) # reshape patches to image-like format
        x = x.reshape(-1, 768, 14, 14)
        # print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.linear1(x)
        x = self.relu3(x)
        x = self.linear2(x)
        print(x.shape)
        return x
    
import torch.nn as nn 
import torch

class Mlp(nn.Module): 
    """
    MLP classifier. 
    Args:
        num_classes -> number of classes 
        in_feature -> features dimension

    return logits. 
    
    """
    def __init__(self,num_classes=2 ,in_features = 768*196):
        
        super().__init__()
        self.linear1 = nn.Linear(in_features= in_features, out_features= 4096)
        self.linear2 = nn.Linear(in_features= 4096, out_features= 2048)
        self.linear3 = nn.Linear(in_features= 2048, out_features= 128)
        self.linear4 = nn.Linear(in_features= 128, out_features= num_classes)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self,x):
        print(x.shape)
        x= x.reshape(-1, 196*768)
        x = nn.functional.relu(self.linear1(x))
        x = nn.functional.relu(self.linear2(x))
        x = nn.functional.relu(self.linear3(x))
        x = self.linear4(x)
        print(x.shape)
        return x

import torch.nn as nn
import torchvision.models as models

class ResNet18_FT(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        print("HI")
        x = self.resnet18.layer1(x)
        print("HI")
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x


class ResNet18_TL(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768, freeze=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)

        # Freeze layers
        if freeze:
            for param in self.resnet18.parameters():
                param.requires_grad = False

        self.resnet18.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x


class ResNet18_FT_CNN(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)
        self.additional_convs = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.resnet18.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.additional_convs(x) # pass through additional conv layers
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x

import torch.nn as nn
import torchvision.models as models

class ResNet18_TL_CNN(nn.Module):
    """
    ResNet18 classifier. 
    Args:
        num_classes -> number of classes 
        in_features -> features dimension
    Return: logits. 
    """
    def __init__(self, num_classes=2, in_features=196*768):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=768, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.resnet18 = models.resnet18(pretrained=True)
        
        # Freeze ResNet layers
        for param in self.resnet18.parameters():
            param.requires_grad = False
        
        self.additional_convs = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.resnet18.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        print(x.shape)
        x = x.reshape(-1, 768, 14, 14) # reshape patches to image-like format
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)
        x = self.additional_convs(x) # pass through additional conv layers
        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.resnet18.fc(x)
        print(x.shape)
        return x


# clf = CNN()
# # print(clf)
# output = clf(x)
# countParams(clf)

# mlp = Mlp()
# output = clf(x)
# countParams(mlp)

# resnet = ResNet18_FT()
# output = resnet(x)
# countParams(resnet)

# resnet = ResNet18_TL()
# output = resnet(x)
# countParams(resnet)

# resnet = ResNet18_FT_CNN()
# output = resnet(x)
# countParams(resnet)

resnet = ResNet18_TL_CNN()
output = resnet(x)
countParams(resnet)

torch.Size([30, 768, 14, 14])
HI
HI
HI
torch.Size([30, 256, 1, 1])


RuntimeError: running_mean should contain 256 elements not 128

# Testing

In [3]:
def test_clfs(mlps_list, data_loader, mlp_root_dir, model):
    length = (len(mlps_list))
    print(length)
    # test_list = {f'block_{i}': [] for i in range(length)}
    # test_acc = 0.0
    # test_loss = 0.0
    # criterion = nn.CrossEntropyLoss()

    for index in range(length): 
        acc_avg = 0.0
        mlp_in = os.path.join(mlp_root_dir, mlps_list[index])
        print(mlp_in)
        print(mlp_in, f": MLP at index {index} has been loaded")
        classifier = torch.load(mlp_in).cuda()
        classifier.eval()
        data_loader_test = data_loader['test']

        with torch.no_grad(): 
            for image, label in tqdm(data_loader_test):
                image = image.cuda()
                label = label.cuda()
                x = model.patch_embed(image)
                x = torch.cat((model.cls_token.expand(x.shape[0], -1, -1), x), dim=1)
                x = model.pos_drop(x + model.pos_embed)
                for i in range(index+1):
                    x = model.blocks[i](x)
                output = classifier(x[:,1:,:])
                predictions = torch.argmax(output, dim=-1)
                acc = torch.sum(predictions == label).item()/len(label)
                acc_avg += acc
            print(f'Accuracy of block {index} = {(acc_avg/len(data_loader_test)):.3f}')
        print(f'================= Block {index} Finished ==============')


### \__main\__

In [5]:
# __main__
# mlp
root_dir = "data/TB_data/"
mlp_path = 'ReVIT/models/mlps'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

mlps_list = sorted(os.listdir(mlp_path))
print(mlps_list)
acc = test_clfs(mlps_list= mlps_list, data_loader=loader_, mlp_root_dir=mlp_path, model=model)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
['mlp_block_0_train_acc0.88_test_acc0.90.pth', 'mlp_block_1_train_acc0.89_test_acc0.92.pth', 'mlp_block_2_train_acc0.92_test_acc0.96.pth', 'mlp_block_3_train_acc0.92_test_acc0.91.pth', 'mlp_block_4_train_acc0.94_test_acc0.95.pth', 'mlp_block_5_train_acc0.50_test_acc0.51.pth', 'mlp_block_6_train_acc0.95_test_acc0.96.pth', 'mlp_block_7_train_acc0.95_test_acc0.95.pth', 'mlp_block_8_train_acc0.95_test_acc0.97.pth', 'mlp_block_910_train_acc0.95_test_acc0.94.pth', 'mlp_block_911_train_acc0.95_test_acc0.95.pth', 'mlp_block_9_train_acc0.95_test_acc0.96.pth']
12
ReVIT/models/mlps/mlp_block_0_train_acc0.88_test_acc0.90.pth
ReVIT/models/mlps/mlp_block_0_train_acc0.88_test_acc0.90.pth : MLP at index 0 has been loaded


100%|██████████| 23/23 [00:02<00:00, 10.92it/s]


Accuracy of block 0 = 0.901
================= Block 0 Finished ==============
ReVIT/models/mlps/mlp_block_1_train_acc0.89_test_acc0.92.pth
ReVIT/models/mlps/mlp_block_1_train_acc0.89_test_acc0.92.pth : MLP at index 1 has been loaded


100%|██████████| 23/23 [00:02<00:00, 10.96it/s]


Accuracy of block 1 = 0.925
================= Block 1 Finished ==============
ReVIT/models/mlps/mlp_block_2_train_acc0.92_test_acc0.96.pth
ReVIT/models/mlps/mlp_block_2_train_acc0.92_test_acc0.96.pth : MLP at index 2 has been loaded


...

Accuracy of block 9 = 0.781
================= Block 9 Finished ==============
ReVIT/models/mlps/mlp_block_911_train_acc0.95_test_acc0.95.pth
ReVIT/models/mlps/mlp_block_911_train_acc0.95_test_acc0.95.pth : MLP at index 10 has been loaded


100%|██████████| 23/23 [00:03<00:00,  5.92it/s]


Accuracy of block 10 = 0.862
================= Block 10 Finished ==============
ReVIT/models/mlps/mlp_block_9_train_acc0.95_test_acc0.96.pth
ReVIT/models/mlps/mlp_block_9_train_acc0.95_test_acc0.96.pth : MLP at index 11 has been loaded


100%|██████████| 23/23 [00:04<00:00,  5.41it/s]

Accuracy of block 11 = 0.957
================= Block 11 Finished ==============


In [6]:
# __main__
# r_mlp
root_dir = "data/TB_data/"
r_mlp_path = 'ReVIT/models/r_mlps'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

r_mlps_list = sorted(os.listdir(r_mlp_path))
print(r_mlps_list)
acc = test_clfs(mlps_list= r_mlps_list, data_loader=loader_, mlp_root_dir=r_mlp_path, model=model)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
['r_mlp_block_0_train_acc0.76_test_acc0.88.pth', 'r_mlp_block_10_train_acc0.90_test_acc0.93.pth', 'r_mlp_block_11_train_acc0.92_test_acc0.91.pth', 'r_mlp_block_1_train_acc0.83_test_acc0.87.pth', 'r_mlp_block_2_train_acc0.85_test_acc0.90.pth', 'r_mlp_block_3_train_acc0.85_test_acc0.90.pth', 'r_mlp_block_4_train_acc0.91_test_acc0.94.pth', 'r_mlp_block_5_train_acc0.92_test_acc0.94.pth', 'r_mlp_block_6_train_acc0.91_test_acc0.95.pth', 'r_mlp_block_7_train_acc0.92_test_acc0.96.pth', 'r_mlp_block_8_train_acc0.95_test_acc0.95.pth', 'r_mlp_block_9_train_acc0.92_test_acc0.96.pth']
12
ReVIT/models/r_mlps/r_mlp_block_0_train_acc0.76_test_acc0.88.pth
ReVIT/models/r_mlps/r_mlp_block_0_train_acc0.76_test_acc0.88.pth : MLP at index 0 has been loaded


100%|██████████| 23/23 [00:01<00:00, 12.32it/s]


Accuracy of block 0 = 0.875
================= Block 0 Finished ==============
ReVIT/models/r_mlps/r_mlp_block_10_train_acc0.90_test_acc0.93.pth
ReVIT/models/r_mlps/r_mlp_block_10_train_acc0.90_test_acc0.93.pth : MLP at index 1 has been loaded


100%|██████████| 23/23 [00:02<00:00, 10.59it/s]


Accuracy of block 1 = 0.507
================= Block 1 Finished ==============
ReVIT/models/r_mlps/r_mlp_block_11_train_acc0.92_test_acc0.91.pth
ReVIT/models/r_mlps/r_mlp_block_11_train_acc0.92_test_acc0.91.pth : MLP at index 2 has been loaded


...

Accuracy of block 9 = 0.514
================= Block 9 Finished ==============
ReVIT/models/r_mlps/r_mlp_block_8_train_acc0.95_test_acc0.95.pth
ReVIT/models/r_mlps/r_mlp_block_8_train_acc0.95_test_acc0.95.pth : MLP at index 10 has been loaded


100%|██████████| 23/23 [00:03<00:00,  5.90it/s]


Accuracy of block 10 = 0.633
================= Block 10 Finished ==============
ReVIT/models/r_mlps/r_mlp_block_9_train_acc0.92_test_acc0.96.pth
ReVIT/models/r_mlps/r_mlp_block_9_train_acc0.92_test_acc0.96.pth : MLP at index 11 has been loaded


100%|██████████| 23/23 [00:04<00:00,  5.66it/s]

Accuracy of block 11 = 0.948
================= Block 11 Finished ==============


In [ ]:
# __main__
# cnns
root_dir = "data/TB_data/"
cnns_path = 'ReVIT/models/cnns'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

cnns_list = sorted(os.listdir(cnns_path))
print(cnns_list)
acc = test_clfs(mlps_list= cnns_list, data_loader=loader_, mlp_root_dir=cnns_path, model=model)

In [10]:
# __main__
# resnets_ft
root_dir = "data/TB_data/"
resnets_ft_path = 'ReVIT/models/resnets_ft'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

resnets_ft_list = sorted(os.listdir(resnets_ft_path))
print(resnets_ft_list)
acc = test_clfs(mlps_list= resnets_ft_list, data_loader=loader_, mlp_root_dir=resnets_ft_path, model=model)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
['resnet_ft_block_0_train_acc0.89_test_acc0.90.pth', 'resnet_ft_block_1_train_acc0.90_test_acc0.93.pth', 'resnet_ft_block_2_train_acc0.91_test_acc0.94.pth', 'resnet_ft_block_3_train_acc0.92_test_acc0.93.pth', 'resnet_ft_block_4_train_acc0.93_test_acc0.96.pth', 'resnet_ft_block_5_train_acc0.95_test_acc0.96.pth', 'resnet_ft_block_6_train_acc0.94_test_acc0.94.pth', 'resnet_ft_block_7_train_acc0.95_test_acc0.95.pth']
8
ReVIT/models/resnets_ft/resnet_ft_block_0_train_acc0.89_test_acc0.90.pth
ReVIT/models/resnets_ft/resnet_ft_block_0_train_acc0.89_test_acc0.90.pth : MLP at index 0 has been loaded


100%|██████████| 23/23 [00:01<00:00, 11.93it/s]


Accuracy of block 0 = 0.904
================= Block 0 Finished ==============
ReVIT/models/resnets_ft/resnet_ft_block_1_train_acc0.90_test_acc0.93.pth
ReVIT/models/resnets_ft/resnet_ft_block_1_train_acc0.90_test_acc0.93.pth : MLP at index 1 has been loaded


100%|██████████| 23/23 [00:02<00:00, 11.29it/s]


Accuracy of block 1 = 0.930
================= Block 1 Finished ==============
ReVIT/models/resnets_ft/resnet_ft_block_2_train_acc0.91_test_acc0.94.pth
ReVIT/models/resnets_ft/resnet_ft_block_2_train_acc0.91_test_acc0.94.pth : MLP at index 2 has been loaded


...

Accuracy of block 5 = 0.962
================= Block 5 Finished ==============
ReVIT/models/resnets_ft/resnet_ft_block_6_train_acc0.94_test_acc0.94.pth
ReVIT/models/resnets_ft/resnet_ft_block_6_train_acc0.94_test_acc0.94.pth : MLP at index 6 has been loaded


100%|██████████| 23/23 [00:03<00:00,  7.11it/s]


Accuracy of block 6 = 0.942
================= Block 6 Finished ==============
ReVIT/models/resnets_ft/resnet_ft_block_7_train_acc0.95_test_acc0.95.pth
ReVIT/models/resnets_ft/resnet_ft_block_7_train_acc0.95_test_acc0.95.pth : MLP at index 7 has been loaded


100%|██████████| 23/23 [00:03<00:00,  6.89it/s]

Accuracy of block 7 = 0.949
================= Block 7 Finished ==============


In [11]:
# __main__
# resnets_ft
root_dir = "data/TB_data/"
resnets_tl_path = 'ReVIT/models/resnets_tl'
vit = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

loader_, dataset_ = data.data_loader(root_dir=root_dir)
model = torch.load(vit).cuda()
model.eval()

resnets_tl_list = sorted(os.listdir(resnets_tl_path))
print(resnets_tl_list)
acc = test_clfs(mlps_list= resnets_tl_list, data_loader=loader_, mlp_root_dir=resnets_tl_path, model=model)

['number of train images is 5670', 'number of valid images is 630', 'number of test images is 700']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
['resnet_tl_block_0_train_acc0.83_test_acc0.88.pth', 'resnet_tl_block_10_train_acc0.93_test_acc0.92.pth', 'resnet_tl_block_11_train_acc0.92_test_acc0.92.pth', 'resnet_tl_block_1_train_acc0.84_test_acc0.83.pth', 'resnet_tl_block_2_train_acc0.85_test_acc0.89.pth', 'resnet_tl_block_3_train_acc0.86_test_acc0.89.pth', 'resnet_tl_block_4_train_acc0.88_test_acc0.94.pth', 'resnet_tl_block_5_train_acc0.90_test_acc0.91.pth', 'resnet_tl_block_6_train_acc0.91_test_acc0.90.pth', 'resnet_tl_block_7_train_acc0.92_test_acc0.93.pth', 'resnet_tl_block_8_train_acc0.92_test_acc0.90.pth', 'resnet_tl_block_9_train_acc0.93_test_acc0.92.pth']
12
ReVIT/models/resnets_tl/resnet_tl_block_0_train_acc0.83_test_acc0.88.pth
ReVIT/models/resnets_tl/resnet_tl_block_0_train_acc0.83_test_acc0.88.pth : MLP at index 0 has been loaded


100%|██████████| 23/23 [00:02<00:00, 11.03it/s]


Accuracy of block 0 = 0.878
================= Block 0 Finished ==============
ReVIT/models/resnets_tl/resnet_tl_block_10_train_acc0.93_test_acc0.92.pth
ReVIT/models/resnets_tl/resnet_tl_block_10_train_acc0.93_test_acc0.92.pth : MLP at index 1 has been loaded


100%|██████████| 23/23 [00:02<00:00, 10.28it/s]


Accuracy of block 1 = 0.507
================= Block 1 Finished ==============
ReVIT/models/resnets_tl/resnet_tl_block_11_train_acc0.92_test_acc0.92.pth
ReVIT/models/resnets_tl/resnet_tl_block_11_train_acc0.92_test_acc0.92.pth : MLP at index 2 has been loaded


...

Accuracy of block 9 = 0.491
================= Block 9 Finished ==============
ReVIT/models/resnets_tl/resnet_tl_block_8_train_acc0.92_test_acc0.90.pth
ReVIT/models/resnets_tl/resnet_tl_block_8_train_acc0.92_test_acc0.90.pth : MLP at index 10 has been loaded


100%|██████████| 23/23 [00:04<00:00,  5.67it/s]


Accuracy of block 10 = 0.509
================= Block 10 Finished ==============
ReVIT/models/resnets_tl/resnet_tl_block_9_train_acc0.93_test_acc0.92.pth
ReVIT/models/resnets_tl/resnet_tl_block_9_train_acc0.93_test_acc0.92.pth : MLP at index 11 has been loaded


100%|██████████| 23/23 [00:04<00:00,  5.37it/s]

Accuracy of block 11 = 0.870
================= Block 11 Finished ==============
